# Artificial Neural Network

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import os
#Changing the directory
os.chdir('/content/drive/My Drive/CPCS 483/HW9')

Mounted at /content/drive


## Importing the libraries

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [6]:
tf.__version__

'2.14.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [7]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [8]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [9]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [11]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [14]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [19]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [20]:
ann = tf.keras. models. Sequential()

### Adding the input layer and the first hidden layer

In [22]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [23]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer, dimenssion of y is 1 (binary), sigmoid allows the probab values if multi softmax

In [24]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [27]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set batch size: comparisons is not one by one between pred and actual epoch = learning through several itreration

In [28]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.7110 - accuracy: 0.6125
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5121 - accuracy: 0.7962
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4663 - accuracy: 0.8069
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4475 - accuracy: 0.8119
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4367 - accuracy: 0.8170
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4295 - accuracy: 0.8199
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4239 - accuracy: 0.8235
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4195 - accuracy: 0.8244
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4153 - accuracy: 0.8265
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4113 - accura

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation (HW Solution)

In [29]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 107ms/step
[[False]]


**Therefore, our ANN model predicts that this customer stays in the bank!**

### Predicting the Test set results

In [33]:
y_pred = ann.predict(sc.transform(X_test))
y_pred = (y_pred > 0.5)
print (np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 1s 8ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [34]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print (cm)
accuracy_score(y_test, y_pred)

[[1518   77]
 [ 195  210]]


0.864